In [3]:
import pandas as pd 
import numpy as np
import os

In [4]:
import mglearn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ModuleNotFoundError: No module named 'mglearn'

In [15]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
from sklearn.model_selection import GridSearchCV

In [16]:
pos= [x for x in os.listdir("train/pos/") if x.endswith(".txt")]
neg= [x for x in os.listdir("train/neg/") if x.endswith(".txt")]
test= [x for x in os.listdir("test/") if x.endswith(".txt")]

In [17]:
# need to add reference or change data processing with data grabber
posReviews=[]
for txt in pos:
    with open("train/pos/"+txt, encoding="ISO-8859-1") as f:
        posReviews.append(f.read())
negReviews=[]        
for txt in neg:
    with open("train/neg/"+txt, encoding="ISO-8859-1") as f:
        negReviews.append(f.read())
testReviews=[]        
for txt in test:
    with open("test/"+txt, encoding="ISO-8859-1") as f:
        testReviews.append(f.read())

In [18]:
# end data processing 
reviews = pd.concat([
    pd.DataFrame({"file":pos,"review":posReviews, "label":1}),
    pd.DataFrame({"file":neg,"review":negReviews, "label":0}),
    pd.DataFrame({"file":test,"review":testReviews, "label":-1})
], ignore_index=True).sample(frac=1, random_state=1)


In [19]:
# Examine firts 10 rows
reviews["file"]= reviews["file"].str.split("_", n = 1, expand = True)
reviews["file"]= reviews["file"].str.split(".", n = 1, expand = True)
#reviews.set_index('file',inplace=True)
reviews.head(10)

,file,review,label
26247,11119,This movie is a desperate attempt to ride the ...,-1
35067,19058,The first time I ever saw this movie was when ...,-1
34590,18629,"This movie will send chills down your spine, e...",-1
16668,2501,I saw this on TV the other nightÂ or rather I...,0
12196,9728,I am a huge fan of Simon Pegg and have watched...,1
2600,12340,There is indeed much to complain about this mo...,1
9047,6894,"The men can slaver over Lollo, if they like (o...",1
2206,11987,Since it has been some years since I reviewed ...,1
25607,10543,"Wow, what exciting visual effects. I also love...",-1
11606,9197,"This is actually a groovy-neat little flick, m...",1


In [20]:
# Examine the class ditribution 
reviews.label.value_counts()

-1    25000
 1    12500
 0    12500
Name: label, dtype: int64

In [21]:
# Define X and y from the review dataset  for use with Countvectorizer
X= reviews[reviews.label!=-1].review
y= reviews[reviews.label!=-1].label
print (X.shape)
print (y.shape)

(25000,)
(25000,)


In [22]:
# Split X and y into training and testing/validation sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(20000,)
(5000,)
(20000,)
(5000,)


In [23]:
# Instanstiate the tfidf transformer
vect = TfidfVectorizer(min_df=4, ngram_range=(1, 2))
X_train_counts = vect.fit_transform(X_train)
trans = TfidfTransformer(smooth_idf=False,norm='l2',use_idf=True)

NameError: name 'nltk' is not defined

In [24]:
#Learn training data vocabulary fit and transform
X_train_dtm= trans.fit_transform(X_train_counts)

In [1]:
X_train_dtm

NameError: name 'X_train_dtm' is not defined

In [2]:
print("Vocabulary size: {}".format(len(vect.vocabulary_)))

NameError: name 'vect' is not defined

In [71]:
# Trasnform testing/validation data using fitted vocabulary 
X_test_dtm= vect.transform(X_test)
X_test_dtm

<5000x164059 sparse matrix of type '<class 'numpy.float64'>'
	with 1367992 stored elements in Compressed Sparse Row format>

In [72]:
# Naives bayes from scikit learn for comparison purposes with scratch model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()


In [73]:
#Train the model
%time nb.fit(X_train_dtm, y_train)

CPU times: user 114 ms, sys: 12.4 ms, total: 126 ms
Wall time: 125 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [74]:
#Make class predictions 
y_pred= nb.predict(X_test_dtm)

In [75]:
# Calculate accuracy
from sklearn import metrics
metrics.accuracy_score(y_test,y_pred)


0.8836

In [76]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred)

array([[2283,  221],
       [ 361, 2135]])

In [77]:
# Logitic regression model
from sklearn.linear_model import LogisticRegression

In [78]:
logmodel=LogisticRegression()

In [79]:
logmodel.fit(X_train_dtm, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [80]:
y_pred_log=logmodel.predict(X_test_dtm)

In [81]:
#Now for the classification task we are going to import classification report
from sklearn.metrics import classification_report


In [82]:
print(classification_report(y_test,y_pred_log))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90      2504
           1       0.90      0.89      0.90      2496

   micro avg       0.90      0.90      0.90      5000
   macro avg       0.90      0.90      0.90      5000
weighted avg       0.90      0.90      0.90      5000



In [83]:
metrics.accuracy_score(y_test,y_pred_log)

0.8968

In [84]:
# Trying K nearest neigbors 
from sklearn.tree import DecisionTreeClassifier

In [85]:
dtree= DecisionTreeClassifier()

In [86]:
dtree.fit(X_train_dtm, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [87]:
y_pred_tree=dtree.predict(X_test_dtm)

In [88]:
print(metrics.accuracy_score(y_test,y_pred_tree))
print(classification_report(y_test,y_pred_tree))
print('\n')
print(metrics.confusion_matrix(y_test,y_pred_tree))

0.659
              precision    recall  f1-score   support

           0       0.66      0.65      0.66      2504
           1       0.66      0.67      0.66      2496

   micro avg       0.66      0.66      0.66      5000
   macro avg       0.66      0.66      0.66      5000
weighted avg       0.66      0.66      0.66      5000



[[1628  876]
 [ 829 1667]]


In [89]:
feature_names = vect.get_feature_names()
print("Number of features: {}".format(len(feature_names)))



Number of features: 164059


In [ ]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(penalty="l2", C=1), param_grid, cv=5)
grid.fit(X_train_dtm, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [91]:
y_pred_grid=grid.predict(X_test_dtm)

In [92]:
print(metrics.accuracy_score(y_test,y_pred_grid))
print(classification_report(y_test,y_pred_grid))
print('\n')
print(metrics.confusion_matrix(y_test,y_pred_grid))

0.9076
              precision    recall  f1-score   support

           0       0.92      0.89      0.91      2504
           1       0.90      0.92      0.91      2496

   micro avg       0.91      0.91      0.91      5000
   macro avg       0.91      0.91      0.91      5000
weighted avg       0.91      0.91      0.91      5000



[[2240  264]
 [ 198 2298]]


In [93]:
# Define X and y from the review dataset  for use with Countvectorizer
y_prime= reviews[reviews.label==-1].review


In [94]:
# Trasnform testing/validation data using fitted vocabulary 
X_prime_dtm= vect.transform(y_prime)

In [95]:
y_predi=grid.predict(X_prime_dtm)

In [96]:
submission = pd.DataFrame()

In [97]:
#submission1["id"] = test_df["id"]
#submission1["labels"] = pred

#submission1=submission1.sort("id")
y_predi

array([0, 1, 1, ..., 0, 0, 0])

In [99]:
submission['Id']= reviews[reviews.label==-1].file

In [100]:
submission['Category']=y_predi

In [101]:
submission.head()

,Id,Category
26247,11119,0
35067,19058,1
34590,18629,1
25607,10543,1
43027,3722,1


In [108]:

submission.Id = pd.to_numeric(submission.Id, errors='coerce')

submission.sort_values(by=['Id'], inplace=True)


In [109]:
submission.to_csv('solution1.csv', index=False)